# The Stationeers Jupyter Notebook

## Smelting

In [13]:
basic_ingots = [
    'Iron',
    'Gold',
    'Copper',
    'Nickel',
    'Silver',
    'Silicon',
    'Lead',
]

alloy_ingots = [
    'Steel',
    'Constantan',
    'Solder',
    'Electrum',
    'Invar',
]

super_alloy_ingots = [
    'Astroloy',
    'Hastelloy',
    'Inconel',
    'Stellite',
    'Waspaloy',
]

Data related to ingot smelting is easily available in 2 .xml files inside the game folder. All we need to do is parse it to get the relevant bits.

In [14]:

import xml.etree.ElementTree as ET

In [15]:
def remove_prefab_affixes(prefabname):
    return prefabname.removeprefix('Item').removesuffix('Ingot')


def parseRecipeXML(xmlpath: str, basic_recipes, alloy_recipes, super_alloy_recipes):
    tree = ET.parse(xmlpath)
    root = tree.getroot()

    for recipedata in root.iter('RecipeData'):
        prefab_name = recipedata.find('PrefabName').text
        recipe_result = remove_prefab_affixes(prefab_name)

        if recipe_result in basic_ingots:
            list_to_add = basic_recipes
        elif recipe_result in alloy_ingots:
            list_to_add = alloy_recipes
        elif recipe_result in super_alloy_ingots:
            list_to_add = super_alloy_recipes
        else:
            # Not an ingot
            continue

        recipe = recipedata.find('Recipe')
        if len(recipe) == 1:
            # Recipe to re-smelt the ingot
            continue

        if recipe_result == 'Steel' and recipe.find('Hydrocarbon') is not None:
            # Alternative recipe
            continue

        recipe_dict = dict(
            name=recipe_result,
            ingredients=dict(),
            smelting_temps=dict(
                unit='K',
            ),
            smelting_press=dict(
                unit='MPa',
            ),
        )

        for child in recipe:
            tag = child.tag
            if tag == 'Temperature':
                recipe_dict['smelting_temps']['min'] = float(child.find('Start').text)
                recipe_dict['smelting_temps']['max'] = float(child.find('Stop').text)
            elif tag == 'Pressure':
                # Divided by 1000 to convert from kPa to MPA
                recipe_dict['smelting_press']['min'] = float(child.find('Start').text) / 1000 
                recipe_dict['smelting_press']['max'] = float(child.find('Stop').text) / 1000
            else:
                recipe_dict['ingredients'][tag] = child.text

        list_to_add.append(recipe_dict)
        

### 

In [16]:
data_folder = '../data'
furnace_xml = 'furnace.xml'
advanced_furnace_xml = 'advancedfurnace.xml'

In [17]:
basic_recipes = []
alloy_recipes = []
super_alloy_recipes = []

parseRecipeXML(data_folder + '/' + furnace_xml, basic_recipes, alloy_recipes, super_alloy_recipes)
parseRecipeXML(data_folder + '/' + advanced_furnace_xml, basic_recipes, alloy_recipes, super_alloy_recipes)

# Sanity check
print('Found', len(basic_recipes) , 'recipes:',basic_recipes)
print('Found', len(alloy_recipes) , 'recipes:',alloy_recipes)
print('Found', len(super_alloy_recipes) , 'recipes:',super_alloy_recipes)

Found 7 recipes: [{'name': 'Silicon', 'ingredients': {'Silicon': '1'}, 'smelting_temps': {'unit': 'K', 'min': 900.0, 'max': 99999.0}, 'smelting_press': {'unit': 'MPa', 'min': 0.1, 'max': 99.999}}, {'name': 'Iron', 'ingredients': {'Iron': '1'}, 'smelting_temps': {'unit': 'K', 'min': 800.0, 'max': 99999.0}, 'smelting_press': {'unit': 'MPa', 'min': 0.1, 'max': 99.999}}, {'name': 'Gold', 'ingredients': {'Gold': '1'}, 'smelting_temps': {'unit': 'K', 'min': 600.0, 'max': 99999.0}, 'smelting_press': {'unit': 'MPa', 'min': 0.1, 'max': 99.999}}, {'name': 'Copper', 'ingredients': {'Copper': '1'}, 'smelting_temps': {'unit': 'K', 'min': 600.0, 'max': 99999.0}, 'smelting_press': {'unit': 'MPa', 'min': 0.1, 'max': 99.999}}, {'name': 'Silver', 'ingredients': {'Silver': '1'}, 'smelting_temps': {'unit': 'K', 'min': 600.0, 'max': 99999.0}, 'smelting_press': {'unit': 'MPa', 'min': 0.1, 'max': 99.999}}, {'name': 'Lead', 'ingredients': {'Lead': '1'}, 'smelting_temps': {'unit': 'K', 'min': 400.0, 'max': 999

### Smelting Requirements

In [18]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly as plt

In [19]:
fig = go.Figure(
        layout=dict(
        title="Smelting Requirements",
        xaxis_title="Temperature (K)",
        yaxis_title="Pressure (MPa)",
        legend_title="Ingots",
        legend=dict(groupclick="toggleitem")
        )
    )

for group_name, items in [('Basics', basic_recipes), ('Alloys', alloy_recipes), ('Super Alloys', super_alloy_recipes)]:
    for smeltable in items:
        temps = smeltable['smelting_temps']
        press = smeltable['smelting_press']
        fig.add_trace(
            go.Scatter(
                x=[temps['min'],temps['max'],temps['max'],temps['min'],temps['min']], 
                y=[press['min'],press['min'],press['max'],press['max'], press['min']], 
                fill="toself",
                mode='lines',
                legendgroup=group_name,
                legendgrouptitle_text=group_name,
                name=smeltable['name']
            )
        )   

fig.show()

### Alloy Reagent Ratios

In [20]:
rows = 2
cols = 5

subplot_titles = [recipe['name'] for recipe in alloy_recipes + super_alloy_recipes]

fig = make_subplots(rows=rows, cols=cols, specs=[[{'type': 'domain'}]*cols]*rows, subplot_titles=subplot_titles)


for k, recipe in enumerate(alloy_recipes + super_alloy_recipes):
    labels = [ingredient for ingredient in recipe['ingredients'].keys()]
    values = [ratio for ratio in recipe['ingredients'].values()]
    i = (k // cols) + 1
    j = k % cols + 1

    fig.add_trace(go.Pie(labels=labels, values=values), row=i, col=j)


fig.update_layout(
        title="Alloy Reagent Ratios",
        legend_title="Reagents",
)
fig.show()

## Phase Change Diagrams
Phase change curves are calculated according to a [message from a dev on the official Stationeers Discord server](https://discord.com/channels/276525882049429515/395912153795788803/1132134871163207851). I defined the necessary functions and hardcoded data below. 

In [21]:
import numpy as np
from functools import partial

In [22]:
def phase_change_temperature(pressure, coef_a, coef_b):
    return np.power(pressure / coef_a, 1.0 / coef_b)

def phase_change_pressure(temperature, coef_a, coef_b):
    return coef_a * np.power(temperature, coef_b)

In [23]:
gases = dict(
    oxygen = dict(
        name = 'Oxygen',
        coefficient_a = 2.6854996004e-11,
        coeficcient_b = 6.49214937325,
        freezing_temp = 56.4,           #K
        boiling_temp = 90,              #K
        min_condensation_press = 6.3,   #kPa
        max_liquid_temp = 162,          #K
    ),
    nitrogen = dict(
        name = 'Nitrogen',
        coefficient_a = 5.5757107833e-7,
        coeficcient_b = 4.40221368946,
        freezing_temp = 40,             #K
        boiling_temp = 75,              #K
        min_condensation_press = 6.3,   #kPa
        max_liquid_temp = 190,          #K
    ),    
    carbon_dioxide = dict(
        name = 'Carbon Dioxide',
        coefficient_a = 1.579573e-26,
        coeficcient_b = 12.195837931,
        freezing_temp = 218,            #K
        boiling_temp = None,            #K
        min_condensation_press = 517,   #kPa
        max_liquid_temp = 265,          #K        
    ),
    volatiles = dict(
        name = 'Volatiles',
        coefficient_a = 5.863496734e-15,
        coeficcient_b = 7.8643601035,
        freezing_temp = 81.6,           #K
        boiling_temp = 112,             #K
        min_condensation_press = 517,   #kPa
        max_liquid_temp = 195,          #K        
    ),
    pollutant = dict(
        name = 'Pollutant',
        coefficient_a = 2.079033884,
        coeficcient_b = 1.31202194555,
        freezing_temp = 173,            #K
        boiling_temp = None,            #K
        min_condensation_press = 1800,  #kPa
        max_liquid_temp = 425,          #K        
    ),     
    water = dict(
        name = 'Water',
        coefficient_a = 3.8782059839e-19,
        coeficcient_b = 7.90030107708,
        freezing_temp = 273,            #K
        boiling_temp = 373,             #K
        min_condensation_press = 6.3,   #kPa
        max_liquid_temp = 643,          #K
    ),    
    nitrous_oxide = dict(
        name = 'Nitrous Oxide',
        coefficient_a = 0.065353501531,
        coeficcient_b = 1.70297431874,
        freezing_temp = 252,            #K
        boiling_temp = None,            #K
        min_condensation_press = 800,   #kPa
        max_liquid_temp = 431,          #K
    ),     
)

In [24]:
fig = go.Figure(        
    layout=dict(
        title="Phase Change Diagrams",
        xaxis_title="Temperature (K)",
        yaxis_title="Pressure (kPa)",
        legend_title="Substances",
    )
)

temperatures = np.array(range(0, 650, 1))

for idx, (gas, values) in enumerate(gases.items()):
    color = plt.colors.qualitative.Plotly[idx]
    gas_curve = partial(phase_change_pressure, coef_a=values['coefficient_a'], coef_b=values['coeficcient_b'])

    def gas_func():
        pressures = gas_curve(temperatures)

        pressures = np.where(temperatures < values['freezing_temp'], 0, pressures)
        pressures = np.where(temperatures > values['max_liquid_temp'], float('inf'), pressures)
        return pressures

    y = gas_func()

    fig.add_trace(
        go.Scatter(
            x=temperatures, 
            y=y,
            mode='lines',
            name=values['name'],
            line=dict(
                color=color,
            ),
            legendgroup=values['name'],
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=[values['freezing_temp'],values['freezing_temp']],
            y=[gas_curve(values['freezing_temp']), np.max(y, initial=0, where=y != float('inf'))],
            mode='lines',
            name=values['name'],
            showlegend=False,
            line=dict(
                color=color,
            ),
            legendgroup=values['name'],
        )
    )

fig.add_hline(y=101, line_width=1, annotation_text="101kPa", annotation_position="top right",)
fig.add_vline(x=298, line_width=1, annotation_text="298K", annotation_position="top right",)


fig.show()